## Proyecto 3

4 GB de audio no es nada

## Descarga de Kaggel

<a href="https://githubtocolab.com/Astolfo2332/HyM_prediction/blob/main/0.1_Analisis_De_Datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

Aqui mete el json de kaggle

In [ ]:
from google.colab import files

files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
'chmod 600 /root/.kaggle/kaggle.json' #Para proteger la información

In [ ]:
!kaggle datasets download -d vbookshelf/respiratory-sound-database
! unzip respiratory-sound-database.zip